# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
#import panda & numpy
import pandas as pd
import numpy as np

# Load customer datasets from an online source
url1 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df_cust = pd.read_csv(url1)

In [ ]:
df_cust.head(5)

In [ ]:
df_cust.info()

In [ ]:
#1 create a new df from the original df keeping only customers with total_claim_amount greater than $1,000 and have a response of "Yes"
df_cust_2 = df_cust[(df_cust["Total Claim Amount"]>=1000) & (df_cust["Response"]=="Yes")]
df_cust_2["Total Claim Amount"] = df_cust_2["Total Claim Amount"].round(2) #rounding total claim amount to 2 decimals for better lisibility
df_cust_2

In [ ]:
df_cust.groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean()

In [ ]:
#2 analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes"
df_cust_yes = df_cust[(df_cust["Response"]=="Yes")]
df_cust_yes.groupby(['Response','Policy Type', 'Gender']).agg({"Total Claim Amount" : ["mean","std"]}).round(2).reset_index()
# comments:
# the mean spreads between lowest 408 for Male + corporate and highest 457 Male personnal  

In [ ]:
#4 Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.
df_cust.groupby (['Education','Gender']).agg({"Customer Lifetime Value" : ["max","min","median"]}).round(2).reset_index()

In [ ]:
#3 Analyze the total number of customers who have policies in each state, and then filter on states where there are more than 500 customers.

In [11]:
#3.1 create a new df containing only cutomers who have a policy in each state
df_cust_all_states=df_cust[['Customer','State']].drop_duplicates()
group_cust_state = df_cust_all_states.groupby('State').agg({'Customer': 'nunique'})
group_cust_state

,Customer
State,
Arizona,1703
California,3150
Nevada,882
Oregon,2601
Washington,798


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     10910 non-null  int64  
 1   Customer                       10910 non-null  object 
 2   State                          10279 non-null  object 
 3   Customer Lifetime Value        10910 non-null  float64
 4   Response                       10279 non-null  object 
 5   Coverage                       10910 non-null  object 
 6   Education                      10910 non-null  object 
 7   Effective To Date              10910 non-null  object 
 8   EmploymentStatus               10910 non-null  object 
 9   Gender                         10910 non-null  object 
 10  Income                         10910 non-null  int64  
 11  Location Code                  10910 non-null  object 
 12  Marital Status                 10910 non-null 